In [45]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

import json

In [4]:
folders = ['train_annotations', 'test_annotations']
files = np.array([])
for folder in folders:
    files = np.append(files, glob(folder + "/*.json"))

In [9]:
corpus = []
labels = []
for file in tqdm(files):
    data = json.load(open(file, 'r'))
    corpus.append([data['form'][i]['text'] for i in range(len(data['form']))])
    labels.append([data['form'][i]['label'] for i in range(len(data['form']))])

100%|██████████| 199/199 [00:00<00:00, 1751.94it/s]


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform([' '.join(c) for c in corpus])
one_hot = pd.get_dummies(labels[0])

In [22]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=4, n_iter=7, random_state=42)

svd.fit(tfidf)
svd.components_

array([[ 9.32208252e-02,  7.49219861e-02,  2.00624503e-03, ...,
         8.96639728e-03,  2.09757786e-03,  1.03793854e-03],
       [ 3.26490129e-02,  5.40672881e-02,  2.76470716e-05, ...,
         2.31952448e-02,  4.51751764e-03,  3.40172113e-04],
       [-8.80517978e-02, -1.42536357e-01, -7.20879953e-05, ...,
         2.02761799e-02,  4.61773580e-03, -3.19222916e-04],
       [ 1.08797488e-02, -6.36892368e-02, -4.54251449e-04, ...,
        -1.39355163e-03, -4.16601903e-04, -5.01762035e-04]])

In [48]:
similarity = cosine_similarity(svd.components_.T)

In [51]:
similarity_mean = similarity.mean()
similarity_std = similarity.std()

In [56]:
import cntk as C
import numpy as np
from tqdm import tqdm
from cntk.losses import binary_cross_entropy, cross_entropy_with_softmax, weighted_binary_cross_entropy

from lib.running_variance import RunningVariance
from itertools import count
from lib import plotting
from lib.envs.bandit import BanditEnv, ActionSpace

C.cntk_py.force_deterministic_algorithms()

import pickle

import scipy

from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import pdist

from abc import ABC, abstractmethod
from collections import defaultdict, namedtuple
import math
from cntk.layers import Sequential, Dense, GRU
from cntk.layers import *
from sklearn.preprocessing import MinMaxScaler
import itertools
import matplotlib.pyplot as plt

NOISE_PARAM = 4.30
MAX_CLUSTERS = 1000

def run_mcts(clusters, noise):
    
    _HCC = namedtuple("HierarchicalCluster", "cluster similarity terminal")

    class HierarchicalCluster(_HCC, ABC):

        @staticmethod
        def has_children(cluster, n):
            cluster_idxs = np.where(clusters==cluster)[0]
            return n == np.min(similarity[cluster_idxs])
        
        def find_children(self):
            cluster_idxs = np.where(clusters==(self.cluster+1))[0]
            return {
                HierarchicalCluster(cluster=clusters[idx], similarity=similarity[idx], 
                    terminal=HierarchicalCluster.set_terminal(
                        clusters[idx], similarity[idx])) for idx in cluster_idxs
            }

        def find_random_child(self):
            cluster_no = np.random.choice(
                list(set(np.arange(1,MAX_CLUSTERS+1,1).tolist()) - set([self.cluster]))
            )
            sim_value = np.random.choice(similarity[clusters==cluster_no])
            return HierarchicalCluster(cluster=cluster_no, similarity=sim_value, 
            terminal=HierarchicalCluster.set_terminal(cluster_no, noise_value))

        @staticmethod
        def set_terminal(cluster, n):
            return cluster == 999 or HierarchicalCluster.has_children(cluster, n)
        
        def is_terminal(self):
            return self.terminal

        def reward(self):
            return (MAX_CLUSTERS - self.cluster) / MAX_CLUSTERS + NOISE_PARAM - \
        abs(scaler.transform(((np.array([self.similarity]*action_count) - similarity_mean) / similarity_std).reshape(1,-1)).flatten()[0])
        
    return HierarchicalCluster

class MCTS:

    def __init__(self, exploration_weight=1, env=None):
        self.Q = defaultdict(int)  # total reward of each node
        self.N = defaultdict(int)  # total visit count for each node
        self.children = dict()  # children of each node
        self.exploration_weight = exploration_weight
        self.env = env

    def choose(self, node):
        if node.is_terminal():
            raise RuntimeError(f"choose called on terminal node {node}")

        if node not in self.children:
            return node.find_random_child()

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score), np.max(list(map(score, self.children[node])))

    def do_rollout(self, node):
        path = self._select(node)
        leaf = path[-1]
        self._expand(leaf)
        reward = self._simulate(leaf)
        self._backpropagate(path, reward)

    def _select(self, node):
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                # node is either unexplored or terminal
                return path
            unexplored = self.children[node] - self.children.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self._uct_select(node)  # descend a layer deeper

    def _expand(self, node):
        if node in self.children:
            return  # already expanded
        self.children[node] = node.find_children()

    def _simulate(self, node):
        invert_reward = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                self.env.is_reset = True
                return 1 - reward if invert_reward else reward
            node = node.find_random_child()
            invert_reward = not invert_reward

    def _backpropagate(self, path, reward):
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = 1 - reward  # 1 for me is 0 for my enemy, and vice versa

    def _uct_select(self, node):
        # All children of node should already be expanded:
        assert all(n in self.children for n in self.children[node])
        
        log_N_vertex = math.log(self.N[node])
        
        def uct(item):
            return self.Q[item] / self.N[item] + self.exploration_weight * math.sqrt(
                log_N_vertex / self.N[item]
            )
        
        return max(self.children[node], key=uct)

def state_space_model(similarity, tfidf):
    def state_space(state):
        sim = similarity[state]
        
        return sim
    
    def non_deterministic_hierarchical_clustering(model, n_clusters=MAX_CLUSTERS):
        sim = similarity[model]
        Z = ward(pdist(sim))
        clusters = fcluster(Z, t=n_clusters, criterion='maxclust')
        
        return clusters
    
    def maximum_similarity_model(model, clusters, env=None):
        sim = similatity[model]
        node = run_mcts(clusters, sim)(cluster=1,similatity=sim[0], terminal=False)
        mcts = MCTS(env=env)
        
        while True:
            for i in range(25):
                mcts.do_rollout(node)
            node, score = mcts.choose(node)
            if node.terminal:
                break
        state_selected = np.where((clusters == node.cluster) & (sim == node.similarity))[0][0]
        return state_selected, score
    
    def marginal_model(action):
        state = tfidf.todense()[:,action]
        
        return state
    
    return state_space, non_deterministic_hierarchical_clustering, maximum_similarity_model, marginal_model

state_space, non_deterministic_hierarchical_clustering, maximum_similarity_model, marginal_model = \
state_space_model(similarity, tfidf)

class PGCREnv(BanditEnv):
    def __init__(self, num_actions = 10, observation_space = None, distribution = "factor_model", evaluation_seed=387):
        super(BanditEnv, self).__init__()
        
        self.action_space = ActionSpace(range(num_actions))
        self.distribution = distribution
        
        self.observation_space = observation_space
        
        np.random.seed(evaluation_seed)
        
        self.reward_parameters = [similarity.sum(axis=1), similarity.sum(axis=1)]
        
        self.optimal_arm = np.argmax(self.reward_parameters)
    
    def reset(self):
        self.is_reset = True
        action = np.random.randint(0,action_count)
        return svd.components_[:,action], action
    
    def compute_gap(self, action):
        if self.distribution == "factor_model":
            gap = np.absolute(self.reward_parameters[0][self.optimal_arm] - self.reward_parameters[0][action])
        elif self.distribution != "normal":
            gap = np.absolute(self.reward_parameters[self.optimal_arm] - self.reward_parameters[action])
        else:
            gap = np.absolute(self.reward_parameters[0][self.optimal_arm] - self.reward_parameters[0][action])
        return gap
    
    def step(self, action):
        self.is_reset = False
        
        valid_action = True
        if (action is None or action < 0 or action >= self.action_space.n):
            print("Algorithm chose an invalid action; reset reward to -inf", flush = True)
            reward = float("-inf")
            gap = float("inf")
            valid_action = False
        
        if self.distribution == "bernoulli":
            if valid_action:
                reward = np.random.binomial(1, self.reward_parameters[action])
                gap = self.reward_parameters[self.optimal_arm] - self.reward_parameters[action]
        elif self.distribution == "normal":
            if valid_action:
                reward = self.reward_parameters[0][action] + self.reward_parameters[1][action] * np.random.randn()
                gap = self.reward_parameters[0][self.optimal_arm] - self.reward_parameters[0][action]
        elif self.distribution == "heavy-tail":
            if valid_action:
                reward = self.reward_parameters[action] + np.random.standard_cauchy()
                gap = self.reward_parameters[self.optimal_arm] - self.reward_parameters[action]        #HACK to compute expected gap
        elif self.distribution == "factor_model":
            if valid_action:
                reward = np.linalg.norm(
                    self.reward_parameters[0][action],1
                ) + \
                np.linalg.norm(
                    self.reward_parameters[1][action],1
                ) * np.random.randn()
        else:
            print("Please use a supported reward distribution", flush = True)
            sys.exit(0)
            
        observation = marginal_model(action)
        
        return (observation, action, reward, self.is_reset, '')

state_dim = tfidf.todense().shape[0] # Dimension of state space
action_count = tfidf.todense().shape[1] # Number of actions
hidden_size = 256 # Number of hidden units
update_frequency = 20

# The policy network maps an observation to a probability of taking action 0 or 1.
observations = C.sequence.input_variable((1,state_dim), np.float32, name="obs")
W1 = C.parameter(shape=(state_dim, hidden_size), init=C.glorot_uniform(), name="W1")
b1 = C.parameter(shape=hidden_size, name="b1")
layer1 = C.relu(C.times(observations, W1) + b1)
W2 = C.parameter(shape=(hidden_size, state_dim), init=C.glorot_uniform(), name="W2")
b2 = C.parameter(shape=state_dim, name="b2")
layer2 = C.times(layer1, W2) + b2
dense = C.softmax(Dense(4))
output = dense

# Label will tell the network what action it should have taken.
label = C.sequence.input_variable((1,state_dim), np.float32, name="label")
# return_weight is a scalar containing the discounted return. It will scale the PG loss.
return_weight = C.sequence.input_variable(1, np.float32, name="weight")

loss = C.reduce_mean(C.log(C.square(output - label) + 1e-4) * return_weight, axis=0, name='loss')

# Build the optimizer
lr_schedule = C.learning_rate_schedule(lr=1e-3, unit=C.UnitType.sample)
m_schedule = C.momentum_schedule(0.99)
vm_schedule = C.momentum_schedule(0.999)
optimizer = C.adam([W1, W2, b1, b2], lr_schedule, momentum=m_schedule, variance_momentum=vm_schedule)

# Create a buffer to manually accumulate gradients
gradBuffer = dict((var.name, np.zeros(shape=var.shape)) for var in loss.parameters if var.name in ['W1', 'W2', 'b1', 'b2'])

def discount_rewards(r, svd_model, gamma=0.999):
    """Take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r,dtype=np.float32)
    running_add = 0
    f_ = 0
    f = np.linalg.norm(svd_model[0],1)
    running_add = f
    for t in reversed(range(0, r.size)):
        if (t < (r.size - 1) and r.size >= 2):
            f_ = np.linalg.norm(svd_model[t+1],1)
            f = np.linalg.norm(svd_model[t],1)
            running_add = running_add + gamma * f_ - f
        running_add = running_add + r[t]
        discounted_r[t] = running_add
    return discounted_r

running_variance = RunningVariance()
reward_sum = 0

max_number_of_episodes = 500
num_epochs = 2

epoch_stats = []
net_actions = []
net_rewards = []
net_scores = []

for epoch in tqdm(range(num_epochs)):
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(max_number_of_episodes),
        episode_rewards=np.zeros(max_number_of_episodes),
        episode_running_variance=np.zeros(max_number_of_episodes),
        episode_scores=np.zeros(max_number_of_episodes),
        losses=np.zeros(max_number_of_episodes))

    env = PGCREnv(num_actions = action_count, observation_space = np.zeros((state_dim,1)))
    
    for episode_number in range(max_number_of_episodes):
        states, rewards, labels, scores = [],[],[],[]
        done = False

        observation, model = env.reset()
        factor_sequence = []
        t = 1
        for action_dim_i in range(action_count):
            done = False
            while not done:
                
                state = np.ascontiguousarray(np.reshape(observation[:,action_dim_i], [1,state_dim]).astype(np.float32))
                states.append(state)

                # Run the policy network and get an action to take.
                # probability with actions
                prob = output.eval(arguments={observations: state})[0][0][0]
                # Sample from the bernoulli output distribution to get a discrete action
                action_selected = prob[np.argmax(prob)]
                is_reset = False
                score = 0.0
                action, score = maximum_similarity_model(
                    model, non_deterministic_hierarchical_clustering(model), env=env
                )
                is_reset = env.is_reset
#                 if prob[np.argmax(prob)] > uniform_prob:
#                     action = np.argmax(prob)
#                     is_reset = True
#                 else:
#                     action = np.random.choice(np.arange(0,state_dim))
                # TODO 2
                # Modify saved labels to encourage the network to increase
                # the probability of the chosen action. This label will be used
                # in the loss function above.
                
                net_actions.append(action)

                z = np.ones((1,action_dim)).astype(np.float32) * 1.25/120
                z[:,action_dim_i] = 0.75
                labels.append(z)
                
                # step the environment and get new measurements
                observation, model, reward, done, _ = env.step(action)

                done = is_reset if is_reset is True else False

                observation = np.ascontiguousarray(observation)

                # Uncomment this line to render the environment
                # env.render()
                
                net_rewards.append(reward)
                net_scores.append(score)

                reward_sum += float(reward)

                # Record reward (has to be done after we call step() to get reward for previous action)
                rewards.append(float(reward))

                factor_sequence.append(similarity[action])

                stats.episode_rewards[episode_number] += reward
                stats.episode_lengths[episode_number] = t
                stats.episode_scores[episode_number] += score

                t += 1

        # Stack together all inputs, hidden states, action gradients, and rewards for this episode
        epr = np.vstack(rewards).astype(np.float32)

        # Compute the discounted reward backwards through time.
        discounted_epr = discount_rewards(epr, factor_sequence)
        
        for state, _label, discounted_reward in tuple(zip(states,labels,discounted_epr)):
            # Keep a running estimate over the variance of of the discounted rewards
            running_variance.add(discounted_reward.sum())

            # Forward pass
            arguments = {observations: state, label: _label, 
                 return_weight: discounted_reward}
            state, outputs_map = loss.forward(arguments, outputs=loss.outputs,
                  keep_for_backward=loss.outputs)

            # Backward pass
            root_gradients = {v: np.ones_like(o) for v, o in outputs_map.items()}
            vargrads_map = loss.backward(state, root_gradients, variables=set([W1, W2, b1, b2]))

            stats.losses[episode_number] += np.mean(loss.eval(arguments)[0])

            for var, grad in vargrads_map.items():
                gradBuffer[var.name] += grad

        # Only update every 20 episodes to reduce noise
        if episode_number % update_frequency == 0:
            grads = {W1: gradBuffer['W1'].astype(np.float32),
                     W2: gradBuffer['W2'].astype(np.float32),
                    b1: gradBuffer['b1'].astype(np.float32),
                    b2: gradBuffer['b2'].astype(np.float32)}
            updated = optimizer.update(grads, update_frequency)

            # reset the gradBuffer
            gradBuffer = dict((var.name, np.zeros(shape=var.shape))
                      for var in loss.parameters if var.name in ['W1', 'W2', 'b1', 'b2'])

            print('Episode: %d. Average reward for episode %f. Variance %f' % (episode_number, reward_sum / update_frequency, running_variance.get_variance()))

            print('Episode: %d. Average loss for episode %f' % (episode_number, stats.losses[episode_number]))

            reward_sum = 0

        stats.episode_running_variance[episode_number] = running_variance.get_variance()
    
    plotting.plot_pgresults(stats)
    epoch_stats.append(stats)


  0%|          | 0/2 [00:00<?, ?it/s]


IndexError: too many indices for array

In [ ]:
seaborn.lineplot(np.arange(0,500),scaled_scores)

In [ ]:
seaborn.lineplot(np.arange(0,500), w_dist)

In [ ]:
seaborn.lineplot(np.arange(0,500), b_dist)

In [ ]:
np.save("timstamp.npy", stats.episode_scores.argsort())

### Beta distribution

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions
units_var = tf.Variable(tf.ones(stats.episode_scores.shape[0]))

def units_per_cell():
    return units_var

x = tf.Variable(tf.ones(stats.episode_scores.shape[0]), shape=stats.episode_scores.shape)
alpha = 0.5
beta = 0.5
dist = scipy.stats.beta(alpha, beta)
lr = 1e-1
scaler = MinMaxScaler(feature_range=(0,1)).fit(stats.episode_scores.reshape(-1,1))
scaled_scores = scaler.transform(stats.episode_scores.reshape(-1,1)).flatten()
scaled_scores.sort()

optimizer = tf.train.GradientDescentOptimizer(lr)

y = MinMaxScaler(feature_range=(0,1)).fit_transform(dist.pdf(np.linspace(1e-3,0.999,500)).reshape(-1,1)).flatten()
y.sort()
y_estimated = scaled_scores * units_per_cell()

loss = tf.reduce_mean(tf.abs(y_estimated - y))

train = optimizer.minimize(loss)
init = tf.global_variables_initializer()
local = tf.local_variables_initializer()
n_iter = 200000

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(local)
    loss_values = []
    train_data = []
    y_vals = []
    captured = []
    for step in tqdm(range(n_iter)):
        _, loss_val, units_val, y_val = sess.run([train, loss, units_var, y_estimated])
        loss_values.append(loss_val)
        y_vals.append(y_val)
        if step % 1000 == 0:
            print(step, loss_val)

W0223 11:35:30.920851 139917768674176 deprecation.py:323] From /home/aswin/anaconda3/envs/enscalo_test/lib/python3.6/site-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
  0%|          | 395/200000 [00:00<4:01:34, 13.77it/s]

0 0.4421673


  1%|          | 1207/200000 [00:00<59:04, 56.09it/s]  

1000 0.39103782


  1%|          | 2245/200000 [00:01<11:16, 292.45it/s]

...

 99%|█████████▉| 198209/200000 [01:32<00:00, 1926.55it/s]

198000 4.584812e-05


100%|█████████▉| 199229/200000 [01:32<00:00, 1975.34it/s]

199000 4.5474473e-05


100%|██████████| 200000/200000 [01:33<00:00, 2147.93it/s]


In [3]:
import numpy as np

np.save("beta_units.npy", units_val)
np.save("scaled_scores.npy", scaled_scores)
np.save("beta.npy", y)

### Weibull distribution

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions
units_var = tf.Variable(tf.ones(stats.episode_scores.shape[0]))

def units_per_cell():
    return units_var

x = tf.Variable(tf.ones(stats.episode_scores.shape[0]), shape=stats.episode_scores.shape)
a = 0.5
c = 0.5
dist = scipy.stats.exponweib(a, c)
lr = 1e-1
scaler = MinMaxScaler(feature_range=(0,1)).fit(stats.episode_scores.reshape(-1,1))
scaled_scores = scaler.transform(stats.episode_scores.reshape(-1,1)).flatten()
scaled_scores.sort()

optimizer = tf.train.GradientDescentOptimizer(lr)

y = MinMaxScaler(feature_range=(0,1)).fit_transform(dist.pdf(np.linspace(1e-3,0.999,500)).reshape(-1,1)).flatten()
y.sort()
y_estimated = scaled_scores * units_per_cell()

loss = tf.reduce_mean(tf.abs(y_estimated - y))

train = optimizer.minimize(loss)
init = tf.global_variables_initializer()
local = tf.local_variables_initializer()
n_iter = 200000

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(local)
    loss_values = []
    train_data = []
    y_vals = []
    captured = []
    for step in tqdm(range(n_iter)):
        _, loss_val, units_val, y_val = sess.run([train, loss, units_var, y_estimated])
        loss_values.append(loss_val)
        y_vals.append(y_val)
        if step % 1000 == 0:
            print(step, loss_val)

  0%|          | 391/200000 [00:00<01:43, 1929.31it/s]

0 0.46582383


  1%|          | 1222/200000 [00:00<01:39, 1989.78it/s]

1000 0.4144374


  1%|          | 2260/200000 [00:01<01:40, 1962.17it/s]

...

 99%|█████████▉| 198224/200000 [01:41<00:00, 1904.12it/s]

198000 4.5616292e-05


100%|█████████▉| 199224/200000 [01:42<00:00, 1881.81it/s]

199000 4.5242647e-05


100%|██████████| 200000/200000 [01:42<00:00, 1946.85it/s]


In [5]:
import numpy as np

np.save("weibull_units.npy", units_val)
np.save("weibull.npy", y)

### Cauchy distribution

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions
units_var = tf.Variable(tf.ones(stats.episode_scores.shape[0]))

def units_per_cell():
    return units_var

x = tf.Variable(tf.ones(stats.episode_scores.shape[0]), shape=stats.episode_scores.shape)
dist = scipy.stats.cauchy()
lr = 1e-1
scaler = MinMaxScaler(feature_range=(0,1)).fit(stats.episode_scores.reshape(-1,1))
scaled_scores = scaler.transform(stats.episode_scores.reshape(-1,1)).flatten()
scaled_scores.sort()

optimizer = tf.train.GradientDescentOptimizer(lr)

y = MinMaxScaler(feature_range=(0,1)).fit_transform(dist.pdf(np.linspace(1e-3,0.999,500)).reshape(-1,1)).flatten()
y.sort()
y_estimated = scaled_scores * units_per_cell()

loss = tf.reduce_mean(tf.abs(y_estimated - y))

train = optimizer.minimize(loss)
init = tf.global_variables_initializer()
local = tf.local_variables_initializer()
n_iter = 200000

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(local)
    loss_values = []
    train_data = []
    y_vals = []
    captured = []
    for step in tqdm(range(n_iter)):
        _, loss_val, units_val, y_val = sess.run([train, loss, units_var, y_estimated])
        loss_values.append(loss_val)
        y_vals.append(y_val)
        if step % 1000 == 0:
            print(step, loss_val)

  0%|          | 393/200000 [00:00<01:42, 1941.89it/s]

0 0.16240594


  1%|          | 1271/200000 [00:00<01:34, 2097.89it/s]

1000 0.113601886


  1%|          | 2346/200000 [00:01<01:36, 2045.15it/s]

...

 99%|█████████▉| 198298/200000 [01:35<00:00, 2023.46it/s]

198000 3.063559e-05


100%|█████████▉| 199233/200000 [01:36<00:00, 2014.18it/s]

199000 3.0480303e-05


100%|██████████| 200000/200000 [01:36<00:00, 2072.09it/s]


In [7]:
import numpy as np

np.save("cauchy_units.npy", units_val)
np.save("cauchy.npy", y)

In [8]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions
units_var = tf.Variable(tf.ones(stats.episode_scores.shape[0]))

def units_per_cell():
    return units_var

x = tf.Variable(tf.ones(stats.episode_scores.shape[0]), shape=stats.episode_scores.shape)
dist = scipy.stats.gamma(0.5)
lr = 1e-1
scaler = MinMaxScaler(feature_range=(0,1)).fit(stats.episode_scores.reshape(-1,1))
scaled_scores = scaler.transform(stats.episode_scores.reshape(-1,1)).flatten()
scaled_scores.sort()

optimizer = tf.train.GradientDescentOptimizer(lr)

y = MinMaxScaler(feature_range=(0,1)).fit_transform(dist.pdf(np.linspace(1e-3,0.999,500)).reshape(-1,1)).flatten()
y.sort()
y_estimated = scaled_scores * units_per_cell()

loss = tf.reduce_mean(tf.abs(y_estimated - y))

train = optimizer.minimize(loss)
init = tf.global_variables_initializer()
local = tf.local_variables_initializer()
n_iter = 200000

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(local)
    loss_values = []
    train_data = []
    y_vals = []
    captured = []
    for step in tqdm(range(n_iter)):
        _, loss_val, units_val, y_val = sess.run([train, loss, units_var, y_estimated])
        loss_values.append(loss_val)
        y_vals.append(y_val)
        if step % 1000 == 0:
            print(step, loss_val)

  0%|          | 402/200000 [00:00<01:38, 2016.67it/s]

0 0.4434813


  1%|          | 1282/200000 [00:00<01:39, 2007.23it/s]

1000 0.39209482


  1%|          | 2386/200000 [00:01<01:34, 2095.11it/s]

...

 99%|█████████▉| 198348/200000 [01:38<00:00, 2255.87it/s]

198000 4.3809985e-05


100%|█████████▉| 199252/200000 [01:38<00:00, 2238.17it/s]

199000 4.3436336e-05


100%|██████████| 200000/200000 [01:38<00:00, 2021.87it/s]


In [9]:
import numpy as np

np.save("gamma_units.npy", units_val)
np.save("gamma.npy", y)

### Rayleigh

In [10]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_probability as tfp
tfd = tfp.distributions
units_var = tf.Variable(tf.ones(stats.episode_scores.shape[0]))

def units_per_cell():
    return units_var

x = tf.Variable(tf.ones(stats.episode_scores.shape[0]), shape=stats.episode_scores.shape)
dist = scipy.stats.rayleigh()
lr = 1e-1
scaler = MinMaxScaler(feature_range=(0,1)).fit(stats.episode_scores.reshape(-1,1))
scaled_scores = scaler.transform(stats.episode_scores.reshape(-1,1)).flatten()
scaled_scores.sort()

optimizer = tf.train.GradientDescentOptimizer(lr)

y = MinMaxScaler(feature_range=(0,1)).fit_transform(dist.pdf(np.linspace(1e-3,0.999,500)).reshape(-1,1)).flatten()
y.sort()
y_estimated = scaled_scores * units_per_cell()

loss = tf.reduce_mean(tf.abs(y_estimated - y))

train = optimizer.minimize(loss)
init = tf.global_variables_initializer()
local = tf.local_variables_initializer()
n_iter = 200000

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(local)
    loss_values = []
    train_data = []
    y_vals = []
    captured = []
    for step in tqdm(range(n_iter)):
        _, loss_val, units_val, y_val = sess.run([train, loss, units_var, y_estimated])
        loss_values.append(loss_val)
        y_vals.append(y_val)
        if step % 1000 == 0:
            print(step, loss_val)

  0%|          | 407/200000 [00:00<01:43, 1925.51it/s]

0 0.19833331


  1%|          | 1292/200000 [00:00<01:34, 2102.98it/s]

1000 0.14831807


  1%|          | 2196/200000 [00:01<01:36, 2039.54it/s]

...

 99%|█████████▉| 198268/200000 [01:37<00:00, 2193.58it/s]

198000 2.7019398e-05


100%|█████████▉| 199368/200000 [01:38<00:00, 2008.09it/s]

199000 2.7011629e-05


100%|██████████| 200000/200000 [01:38<00:00, 2033.45it/s]


In [11]:
import numpy as np

np.save("rayleigh_units.npy", units_val)
np.save("rayleigh.npy", y)

In [ ]:
model.summary()

In [ ]:
!pip install --user tensorflow==2.0.0-rc0

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(200),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=22, input_shape=(1,1,24)),
    tf.keras.layers.Activation('softmax')
])

lr = 1e-3
num_epochs = 1000
num_batch = int(120000 / (10*500))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

init = tf.global_variables_initializer()
local = tf.local_variables_initializer()

np_scores = np.array(net_scores)
np_actions = np.array(net_actions)

test_size = 0.25

a = np.arange(120000)
np.random.shuffle(a)
test_choice = a[:int(test_size*120000)]
train_choice = np.array(list(set(np.arange(0,len(np_scores),1).tolist()) - set(test_choice.tolist())))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(local)

    model.fit(np_actions[train_choice].reshape(1,1,-1).astype(np.float64), 
              np_scores[train_choice].reshape(1,1,-1),
          validation_data=(np_actions[test_choice].reshape(1,1,-1).astype(np.float64), 
           np_scores[test_choice].reshape(1,1,-1)),
          batch_size=num_batch, epochs=num_epochs)

In [ ]:
import seaborn

seaborn.lineplot(np.arange(0,500),weibull_units)

In [ ]:
import seaborn

seaborn.lineplot(np.arange(0,500),beta_units)

In [ ]:
seaborn.lineplot(np.arange(0,500),y_val)

In [ ]:
seaborn.lineplot(np.arange(0,500),scaled_scores)

In [ ]:
seaborn.lineplot(np.arange(0,500),y)

In [ ]:
import numpy as np

weibull_units = np.load("weibull_units.npy")
beta_units = np.load("beta_units.npy")

In [ ]:
from pandas.plotting import scatter_matrix, lag_plot
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def create_model(features):
    l1 = layer1
    l2 = layer2
    l3 = layer3
    layer4 = layer3(layer2)
    o = output
    return o(features)

z = create_model(C.sequence.input_variable((120,1), np.float32, name="obs"))
z.save("policy.onnx", format=C.ModelFormat.ONNX)

In [ ]:
plt.plot(np.arange(len(list(losses.values()))), list(losses.values()))